In [4]:
import pandas as pd
import numpy as np

nordpool = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Analise-Sincronia-SeriesTemporais-Soja\NP.csv', usecols=['Price'], ['Date'])

nordpool

SyntaxError: positional argument follows keyword argument (2048560840.py, line 4)